# Etudes des valeurs foncières Française
Liens: https://www.data.gouv.fr/fr/datasets/demandes-de-valeurs-foncieres/

In [ ]:
# Gather Latitude Longitude from Google Map (time/cost consuming)
add_lat_long = False

In [ ]:
import numpy as np
import pandas as pd
import missingno as msno
import glob
from datetime import datetime

## Chargement du fichiers des valeurs foncières

In [ ]:
dateparse = lambda x: datetime.strptime(x, '%d/%m/%Y') # 03/01/2018
def_types = {'Code postal': 'category',
             'B/T/Q': 'category',
             'Nature mutation': 'category',
             'Type de voie': 'category',
             'Voie': 'category',
             'Commune': 'category',
             'Code departement': 'category',
             'Type local': 'category',
             'Nature culture': 'category',
             'Nature culture speciale': 'category',
             
             'No voie': 'Int64',
             'Surface Carrez du 1er lot':'Float16',
             'Surface Carrez du 2eme lot':'Float16',
             'Surface Carrez du 3eme lot':'Float16',
             'Surface Carrez du 4eme lot':'Float16',
             'Surface Carrez du 5eme lot':'Float16',
             'Nombre de lots':'Int16',
             'Nombre de lots':'Int16',
             'Surface reelle bati': 'Float32',
             'Nombre pieces principales': 'Int8',
             'Surface terrain':'Float32'
             #'Date mutation':'datetime64'
            }
col_exclude = ['Prefixe de section', 'Section', 'No plan',
               'No Volume','No disposition','Code type local', 'Code voie', 
               '5eme lot', '4eme lot', '3eme lot', '2eme lot', '1er lot']
all_files = glob.glob("data/*.txt")
df = pd.concat((pd.read_csv(f, sep='|', decimal=",", dtype=def_types, parse_dates=['Date mutation'], date_parser=dateparse, usecols=lambda col: col not in col_exclude) for f in all_files))
#df = pd.read_csv('data/valeursfoncieres-2018.txt', sep='|', decimal=",", dtype=def_types, parse_dates=['Date mutation'], date_parser=dateparse)

## Suppression des colonnes sans intérêt pour notre étude

In [ ]:
#df.drop(['voie', 'b/t/q'],axis=1, inplace=True)

# Référence cadastrale de la parcelle
df.drop(['Prefixe de section', 'Section', 'No plan', 'No Volume'],axis=1, inplace=True, errors='ignore') # Garde 'Code commune'

# No de disposition
df.drop(['No disposition'],axis=1, inplace=True, errors='ignore')

# Nature culture (référence à un document externe)
#df.drop(['nature_culture_speciale', 'nature_culture'],axis=1, inplace=True)

# Suppression du 'Type local' en doublon avec le 'Code Type Local'
df.drop(['Code type local'],axis=1, inplace=True, errors='ignore')

# Suppression du 'Code voie'
df.drop(['Code voie'],axis=1, inplace=True, errors='ignore')

# Suppression du 'No voie'
#df.drop(['no_voie'],axis=1, inplace=True)

# Suppression de la 'Date mutation'
#df.drop(['date_mutation'],axis=1, inplace=True)

# Suppression de la 'Code commune' en doublon avec la categorie 'Commune'
#df.drop(['code_commune'],axis=1, inplace=True)
# Suppression de 'Commune' en doublon avec Commune
#df.drop(['code_postal'],axis=1, inplace=True)

# Suppression des 'lot'
df.drop(['5eme lot'],axis=1, inplace=True, errors='ignore')
df.drop(['4eme lot'],axis=1, inplace=True, errors='ignore')
df.drop(['3eme lot'],axis=1, inplace=True, errors='ignore')
df.drop(['2eme lot'],axis=1, inplace=True, errors='ignore')
df.drop(['1er lot'],axis=1, inplace=True, errors='ignore')

## Supression des colonnes sans données

In [ ]:
# Drop NaN columns
orig_col = df.columns
df.dropna(how='all', axis=1, inplace=True)
print("Suppression de {} colonnes sans données:".format(len(orig_col)-len(df.columns)))
print([item for item in orig_col if item not in df.columns])

In [ ]:
# Suppression des lignes où il n'y a pas de 'valeurs foncières'
indexNames = df[df['Valeur fonciere'].isnull()].index
df.drop(indexNames, inplace=True)

## Suppression des doublons

In [ ]:
#df[df.duplicated(keep=False)]

In [ ]:
# Drop duplicates
initial_length = len(df)
df.drop_duplicates(inplace=True)
print("Suppression des doublons: {}".format(initial_length-len(df)))

#### Renomme les colonnes

In [ ]:
import unidecode

columns = {}
for index, column_name in enumerate(df.columns):
    columns[column_name] = unidecode.unidecode(column_name.replace(' ','_').replace('\'','_').lower())
df.rename(columns=columns, inplace=True)

#### Analyse

In [ ]:
df.describe()

In [ ]:
df.info(verbose=False, memory_usage="deep")

## Nettoyage des données simple

In [ ]:
def missing_values_assessment(df):
    nan_values = df.isnull().sum().sum()
    print('Nombre d\'observations: {:,}'.format(len(df)))
    print('Nombre de valeurs: {:,}'.format(df.size))
    print('Valeurs manquantes: {:,}'.format(nan_values))
    print('Qualité des données: {}%'.format(100-round((nan_values/df.size)*100,2)))
    print('Type de données:\n {}%'.format(df.dtypes.value_counts()))
    analysis = {'Manquant': df.isnull().sum(),
                'Manquant %':round((df.isnull().sum()/len(df))*100, 2),
                'Type':df.dtypes
               }
    return pd.DataFrame(analysis)

#### Analyse des données manquantes

In [ ]:
missing_values_assessment(df).sort_values('Manquant %', ascending=False)

In [ ]:
# Remplissage des mètres Carrez manquant avec zéro
#df = df[df['Nombre de lots']<=5].dropna(how='all', axis=1)
df['surface_carrez_du_5eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_4eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_3eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_2eme_lot'].fillna(0, inplace=True)
df['surface_carrez_du_1er_lot'].fillna(0, inplace=True)
df['surface_reelle_bati'].fillna(0, inplace=True)
df['surface_terrain'].fillna(0, inplace=True)
df['nombre_pieces_principales'].fillna(0, inplace=True)

#### Analyse graphique des données manquantes

In [ ]:
msno.bar(df, sort='ascending')

In [ ]:
msno.matrix(df, sort='ascending')

## Nettoyage de données fonctionnelles

In [ ]:
# Colonne 'B/T/Q'
df.loc[df['b/t/q'].isin([',','/','*',"'",'-','.']), ['b/t/q']] = np.NaN

In [ ]:
df['code_departement'].dtypes

In [ ]:
df['code_departement'].dtype

In [ ]:
# Colonne Code Département
df['code_departement'] = df['code_departement'].astype('str')
df.loc[df['code_departement']=='2A', ['code_departement']] = '120'
df.loc[df['code_departement']=='2B', ['code_departement']] = '121'
df['code_departement'] = df['code_departement'].astype(np.int16) # Converti en int16 pour la mémoire

In [ ]:
# Colonne code commune
df['code_commune'] = df['code_commune'].astype(np.int16)

In [ ]:
## Formatage du code postal
df.code_postal=df.code_postal.apply(lambda x: '0{}'.format(x) if len(str(x))==4 else x)

In [ ]:
category_cols = df.select_dtypes(include=['object','category'])

In [ ]:
category_cols

In [ ]:
for cat in category_cols.columns:
    cat_val = df[cat].unique()
    print('{} ({}):'.format(cat,len(cat_val)))
    print('{}\n'.format(cat_val))

In [ ]:
df.info()

### Feature engineering Address

In [ ]:
df['address']=(df['no_voie'].map(str).replace('nan','') +  ' ' + \
    df['b/t/q'].fillna('') +  ' ' + \
    df['type_de_voie'].fillna('') +  ' ' + \
    df['voie'].fillna('') +  ', ' + \
    df['code_postal'].astype('str').replace('nan','') +  ' ' + \
    df['commune'].fillna('')).str.strip()

In [ ]:
# Supprime les colonnes qu'on ne réutilisera pas
df.drop(['no_voie', 'b/t/q', 'voie', 'commune'],axis=1, inplace=True)

In [ ]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import GoogleV3

cache_file = 'data/cache.dat'

def save_locations(df):
    df.to_csv(cache_file, index = None, header=True)

def load_locations():
    try:
        return pd.read_csv(cache_file)
    except:
        print('{} doesn\'t exist or is empty - creating a new dataframe'.format(cache_file))
        return pd.DataFrame(columns=['latitude','longitude','address', 'address_unformatted'])

with open('google_api_key.txt') as f:
    google_api_key = f.readline()

def add_locations(df, location, address):
    return df.append({'latitude':location.latitude,
               'longitude':location.longitude,
               'address':location.address,
                'address_unformatted':address},
              ignore_index=True)
    
def get_geocode(df, address):
    location = df[df['address_unformatted']==address]
    if not location.empty :
        return df, location
    #print('Address not in cache, requesting GeoPy')
    geopy = GoogleV3(api_key=google_api_key)
    try:
        location = geopy.geocode(address)
        df = add_locations(df, location, address)
    except Exception as e:
        print('Exception on address: {}'.format(address))
        traceback.print_exc()
    return df, location

In [ ]:
if add_lat_long: 
    df['latitude'] = np.nan
    df.latitude = df.latitude.astype('float')
    df['longitude'] = np.nan
    df.longitude = df.longitude.astype('float')
    df['address_formatted'] = np.nan

In [ ]:
if add_lat_long:
    for index, row in df.iterrows():
        address = row['address']
        #print('Searching address:{}'.format(row['address']))
        geolocation_df, location = get_geocode(geolocation_df, address)
        try:
            if location is not None:
                df.at[index, 'latitude'] = location.latitude
                df.at[index, 'longitude'] = location.longitude
                df.at[index, 'address_formatted'] = location.address
        except Exception as e:
            print('Exception on address [{}]: {}'.format(index, row['address']))
            traceback.print_exc()
        if index%100==0:
            print('Saving - index: {}'.format(index))
            save_locations(geolocation_df)

In [ ]:
missing_values_assessment(df).sort_values('Manquant %', ascending=False)

### Catégories

In [ ]:
df = pd.get_dummies(df, columns=['nature_mutation'])
df = pd.get_dummies(df, columns=['b/t/q'])
#df = pd.get_dummies(df, columns=['type_de_voie'])
df = pd.get_dummies(df, columns=['type_local'])
df = pd.get_dummies(df, columns=['nature_culture'])
df = pd.get_dummies(df, columns=['nature_culture_speciale'])

#### Ajout des données INSEE

In [ ]:
df = df.merge(df_commune, how='left', left_on=['Code postal'], right_on=['Code Postal'])

In [ ]:
# Clean up a bit before merging
df_commune.drop(['Statut','geo_shape','ID Geofla','Code Commune','Code Département','Département','Région', 'Commune', 'Code INSEE'],axis=1, inplace=True)

In [ ]:
file_insee = 'data/correspondance-code-insee-code-postal.csv'
df_commune = pd.read_csv(file_insee, sep=';', decimal=".")

## Save the file

In [ ]:
df.to_csv(r'data/valeursfoncieres-clean-df.csv', index = None, header=True)

In [ ]:
df['code_departement'] = df['code_departement'].astype(str)
df["code_commune"] = df['code_commune'].astype(str)
df['code_departement'] = df['code_departement'].apply(lambda x: '0'+x if len(x)==1 else x)
df['code_commune'] = df['code_commune'].apply(lambda x: x+'0' if len(x)==2 else x)
df['code_commune'] = df['code_commune'].apply(lambda x: x+'00' if len(x)==1 else x)

In [ ]:
df["code_commune_long"] = df["code_departement"] + df["code_commune"]

In [ ]:
df['code_commune_long'].sort_values

In [ ]:
from numpy import savetxt
code_commune_long = df['code_commune_long'].unique()

### Suppression des données  plus utilisés

In [ ]:
# Nous n'avons plus besoin du code postal car nous avons le code commune et le code département
df.drop(['code_postal'],axis=1, inplace=True)


In [ ]:
savetxt('data/code_commune.csv', code_commune_long, delimiter=',', fmt='"%s"')